# `TimeSeriesDataset` Sample

In [2]:
import numpy as np
import pandas as pd
from pytorch_forecasting import TimeSeriesDataSet

In [3]:
sample_data = pd.DataFrame(
    dict(
        time_idx=np.tile(np.arange(6), 3),
        target=np.array([0,1,2,3,4,5,20,21,22,23,24,25,40,41,42,43,44,45]),
        group=np.repeat(np.arange(3), 6),
        holidays = np.tile(['X','Black Friday', 'X','Christmas','X', 'X'],3),
    )
)
sample_data

,time_idx,target,group,holidays
0,0,0,0,X
1,1,1,0,Black Friday
2,2,2,0,X
3,3,3,0,Christmas
4,4,4,0,X
5,5,5,0,X
6,0,20,1,X
7,1,21,1,Black Friday
8,2,22,1,X
9,3,23,1,Christmas


In [4]:
# create the time-series dataset from the pandas df
dataset = TimeSeriesDataSet(
    sample_data,
    group_ids=["group"],
    target="target",
    time_idx="time_idx",
    max_encoder_length=2,  # Lookback period
    max_prediction_length=3,  # how many datapoints will be predicted
    time_varying_unknown_reals=["target"],
    static_categoricals=["holidays"],
    target_normalizer=None
)

In [5]:
# pass the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=1)

In [10]:
#load the first batch
x, y = next(iter(dataloader))
print(x['encoder_target'])
print(x['groups'])
print('\n')
print(x['decoder_target'])

tensor([[40., 41.]])
tensor([[2]])


tensor([[42., 43., 44.]])


In [11]:
x

{'encoder_cat': tensor([[[2],
          [0]]]),
 'encoder_cont': tensor([[[40.],
          [41.]]]),
 'encoder_target': tensor([[40., 41.]]),
 'encoder_lengths': tensor([2]),
 'decoder_cat': tensor([[[2],
          [1],
          [2]]]),
 'decoder_cont': tensor([[[42.],
          [43.],
          [44.]]]),
 'decoder_target': tensor([[42., 43., 44.]]),
 'decoder_lengths': tensor([3]),
 'decoder_time_idx': tensor([[2, 3, 4]]),
 'groups': tensor([[2]]),
 'target_scale': tensor([[0., 1.]])}

In [12]:
y

(tensor([[42., 43., 44.]]), None)

# Demand Energy Forecasting Tutorial

### Data Preprocessing

In [17]:
data = pd.read_csv('LD2011_2014.txt', index_col=0, sep=';', decimal=',')
data.index = pd.to_datetime(data.index)
data.sort_index(inplace=True)
data.head(5)

,MT_001,MT_002,MT_003,MT_004,MT_005,MT_006,MT_007,MT_008,MT_009,MT_010,...,MT_361,MT_362,MT_363,MT_364,MT_365,MT_366,MT_367,MT_368,MT_369,MT_370
2011-01-01 00:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 00:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-01 01:15:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
